<a href="https://colab.research.google.com/github/misanchz98/bitcoin-direction-prediction/blob/main/03_modeling/03_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 Modelos de Deep Learning para Series Temporales con Walk-Forward Validation

En este notebook implementamos diferentes modelos de **Deep Learning** y técnicas de validación temporal con **Purged Walk-Forward Split**.  

Incluye:
- Preprocesamiento y creación de secuencias.
- Modelos: LSTM, GRU, CNN-LSTM, Transformer y TCN.
- Métricas personalizadas y evaluación.
- Importancia de características.
- Pipeline de entrenamiento y validación.

## 🔹 1. Librerías
Instalamos e importamos las librerías necesarias para manipulación de datos, visualización, machine learning y deep learning.


In [ ]:
# =============================================================================
# LIBRERIAS
# =============================================================================
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import tensorflow as tf

import warnings

# Establecer el nivel de advertencias a "ignore" para ignorar todas las advertencias
warnings.filterwarnings("ignore")

## 🔹 2. Módulos

In [ ]:
# 1️⃣ REINICIO COMPLETO - Ir al directorio raíz y limpiar todo
import os
import shutil

# Volver al directorio raíz de Colab
os.chdir('/content')

# Eliminar CUALQUIER rastro del repositorio
for item in os.listdir('/content'):
    if 'bitcoin' in item.lower():
        try:
            if os.path.isdir(item):
                shutil.rmtree(item)
            else:
                os.remove(item)
            print(f"Eliminado: {item}")
        except:
            pass

# Verificar que estamos en /content y está limpio
print(f"Directorio actual: {os.getcwd()}")
print("Contenido actual:")
!ls -la

In [ ]:
# 2️⃣ CLONAR FRESCO
!git clone https://github.com/misanchz98/bitcoin-direction-prediction.git

# Verificar que se clonó correctamente
print("Verificando estructura:")
!ls -la bitcoin-direction-prediction/

In [ ]:
# 3️⃣ NAVEGAR CORRECTAMENTE
import sys
import os

# Cambiar al directorio de modeling
os.chdir('/content/bitcoin-direction-prediction/03_modeling')

# Verificar que estamos en el lugar correcto
print(f"Directorio actual: {os.getcwd()}")
print("Contenido:")
!ls -la

# Verificar que existe utils/
print("\nContenido de utils/:")
!ls -la utils/

In [ ]:
# 4️⃣ IMPORTAR CORRECTAMENTE
import sys
import os

# Agregar el directorio actual al path
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.insert(0, current_dir)

# Importar la función
try:
    from utils.pipelines import run_pipeline_random_search
    print("✅ ¡Importación exitosa!")
    print(f"Función disponible: {run_pipeline_random_search}")
except Exception as e:
    print(f"❌ Error: {e}")

    # Plan B - Importación directa
    import importlib.util

    pipeline_path = os.path.join(current_dir, 'utils', 'pipelines.py')
    print(f"Intentando cargar desde: {pipeline_path}")

    if os.path.exists(pipeline_path):
        spec = importlib.util.spec_from_file_location("pipelines", pipeline_path)
        pipelines_module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(pipelines_module)
        run_pipeline_random_search = pipelines_module.run_pipeline_random_search
        print("✅ Plan B exitoso - función cargada!")
    else:
        print("❌ El archivo no existe")

## 🔹 3. Semillas
Para garantizar **reproducibilidad** en los experimentos, es importante fijar las semillas de las librerías que generan números aleatorios:

- `os.environ['PYTHONHASHSEED']` → controla el hash en Python.  
- `numpy.random.seed` → asegura resultados reproducibles en operaciones de NumPy.  
- `random.seed` → fija la semilla del generador de números aleatorios nativo de Python.  
- `tf.random.set_seed` → fija la semilla para TensorFlow y Keras.  

Esto ayuda a que los modelos se entrenen con resultados consistentes entre ejecuciones.

In [ ]:
def reset_random_seeds(seed=42):
    # 1. Python built-in random
    random.seed(seed)

    # 2. NumPy random
    np.random.seed(seed)

    # 3. TensorFlow random
    tf.random.set_seed(seed)

    # 4. Hash seed para operaciones de Python
    os.environ['PYTHONHASHSEED'] = str(seed)

    # 5. Operaciones determinísticas
    os.environ['TF_DETERMINISTIC_OPS'] = '1'

    # 6. Configurar determinismo (con manejo de errores)
    try:
        tf.config.experimental.enable_op_determinism()
        print("✅ Determinismo habilitado")
    except Exception as e:
        print(f"⚠️ enable_op_determinism no disponible: {e}")

    print(f"✅ Seeds configuradas con valor: {seed}")
reset_random_seeds()

## 🔹 4. Conjunto de Datos
Importamos el conjunto de datos en nuestro entorno de trabajo. Se encuentran almacenados en un archivo CSV llamado `btc_historical_data_eda.csv`, cuya obtención se explica en el *notebook* `02_data_analysis.ipynb`.

In [ ]:
# Importamos CSV
url = 'https://raw.githubusercontent.com/misanchz98/bitcoin-direction-prediction/main/02_data_analysis/data/btc_historical_data_eda.csv'
df_bitcoin = pd.read_csv(url, parse_dates=['Open time'])
df_bitcoin

## 🔹 5. Ajuste de Hiperparámetros


Antes de realizar el ajuste de hiperparámetros de las redes neuronales:

1. Establecemos la columna `Open time` como índice del conjunto de datos.
2. Creamos la columna `Return`:

```python
df_bitcoin["Return"] = np.log(df_bitcoin["Close"] / df_bitcoin["Close"].shift(1)).fillna(0)
```

  - **¿Qué significa?**

    - `df_bitcoin["Close"]` → Precio de cierre del Bitcoin.
    - `df_bitcoin["Close"].shift(1)` → Precio de cierre del día anterior.
    - `df_bitcoin["Close"] / df_bitcoin["Close"].shift(1)` → Retorno simple diario:

  - **Fórmula del retorno simple**:
$$
\text{Retorno simple} = \frac{P_t}{P_{t-1}}
$$

    - `np.log(...)` → Retorno logarítmico, que tiene varias ventajas:
      - Es **aditivo en el tiempo**: la suma de retornos logarítmicos de varios días es igual al log del retorno total.
      - Maneja mejor la **volatilidad** y los efectos de la **capitalización continua**.

    - `.fillna(0)` → El primer día no tiene precio anterior, se rellena con 0 para evitar valores nulos.


  - **¿Para qué se usa?** En nuestro pipeline de trading:

    - `Return` no es la variable objetivo (`y`), sino información adicional.
    - Se utiliza para calcular métricas financieras, principalmente el **Sharpe ratio**.
    - Dependiendo de la estrategia (`longonly`, `longshort`, `shortonly`), se combina con las predicciones `y_pred` para simular **ganancias o pérdidas**.


In [ ]:
# Asegúrate que tu DataFrame tenga 'Target' y 'Return'

# Establecemos columna 'Open time' como indice
df_bitcoin["Open time"] = pd.to_datetime(df_bitcoin["Open time"])
df_bitcoin = df_bitcoin.set_index("Open time")

# Creamos la columna Return
df_bitcoin["Return"] = np.log(df_bitcoin["Close"] / df_bitcoin["Close"].shift(1)).fillna(0)

### Estrategia LONGONLY
- Qué hace: Solo toma posiciones largas (compra).
- Objetivo: Ganar cuando el precio sube.
- Ejemplo: Compras BTC esperando que suba, y vendes más caro.
- Ventaja: Menor riesgo que el shorting, especialmente en mercados alcistas.
- Limitación: No se beneficia de caídas del mercado.

In [102]:
# Ejecutar la búsqueda - longonly
results_longonly = run_pipeline_random_search(
    df_bitcoin,
    target_col="Target",
    return_col="Return",
    window_size=30,
    horizon=1,
    test_size=0.2,
    strategy="longonly",   # longonly | shortonly | longshort
    scoring="sharpe",      # sharpe | accuracy | f1 | ...
    n_iter=5               # número de combinaciones a probar
)

Configuración del experimento:
  Estrategia: longonly
  Métrica: sharpe
  Iteraciones: 5
Ejecutando Random Search con 5 combinaciones y 4 modelos...
Estrategia de trading: longonly
Métrica de optimización: sharpe

Combinación 1/5: {'learning_rate': 0.0001, 'batch_size': 16, 'epochs': 50, 'dropout': 0.2, 'lstm_units': 32, 'cnn_filters': 16, 'kernel_size': 2}
  Evaluando LSTM (1/20)
    Score: 0.6640 ± 0.7118 (n_folds=3)
  Evaluando GRU (2/20)
    Score: 0.8118 ± 0.6763 (n_folds=3)
  Evaluando LSTM_CNN (3/20)
    Score: 0.7789 ± 0.1973 (n_folds=3)
  Evaluando GRU_CNN (4/20)
    Score: 0.6961 ± 0.9947 (n_folds=3)

Combinación 2/5: {'learning_rate': 0.0001, 'batch_size': 64, 'epochs': 50, 'dropout': 0.3, 'lstm_units': 32, 'cnn_filters': 64, 'kernel_size': 3}
  Evaluando LSTM (5/20)
    Score: 0.6650 ± 0.9452 (n_folds=3)
  Evaluando GRU (6/20)
    Score: 0.7445 ± 1.1442 (n_folds=3)
  Evaluando LSTM_CNN (7/20)
    Score: 0.4533 ± 0.7459 (n_folds=3)
  Evaluando GRU_CNN (8/20)
    Score: 1.164

KeyboardInterrupt: 

In [ ]:
# Obtener un modelo específico
best_model_lstm_longonly = results_longonly.get_model_by_id("LSTM_comb4")
best_model_gru_longonly = results_longonly.get_model_by_id("GRU_comb4")
best_model_cnn_lstm_longonly = results_longonly.get_model_by_id("LSTM_CNN_comb5")
best_model_cnn_gru_longonly = results_longonly.get_model_by_id("GRU_CNN_comb5")

In [ ]:
# Diccionario de modelos ya seleccionados
models_dict = {
    "LSTM": best_model_lstm_longonly,
    "GRU": best_model_gru_longonly,
    "LSTM_CNN": best_model_cnn_lstm_longonly,
    "GRU_CNN": best_model_cnn_gru_longonly
}

# Evaluar con Purged Walk-Forward SOLO en longonly
results_df = pipeline_evaluate_models(df, models_dict, strategy="longonly")

# Resumen por modelo
summary = results_df.groupby("model").mean().reset_index()
print(summary[["model", "accuracy", "f1", "sharpe", "cum_return"]])

# Boxplot comparativo
plot_model_comparison(results_df, metric="sharpe", strategy="longonly")

### Estrategia LONGSHORT
- Qué hace: Puede tomar ambas posiciones, largas y cortas.
- Objetivo: Aprovechar tanto subidas como bajadas.
- Ejemplo: Compras BTC y vendes ETH si crees que BTC va a subir y ETH a bajar.
- Ventaja: Más flexible, puede generar ganancias en cualquier dirección del mercado.
- Limitación: Más compleja, requiere buena gestión de riesgo.

In [ ]:
# Ejecutar la búsqueda - longshort
results_longshort = run_pipeline_random_search(
    df_bitcoin,
    target_col="Target",
    return_col="Return",
    window_size=30,
    horizon=1,
    test_size=0.2,
    strategy="longshort",   # longonly | shortonly | longshort
    scoring="sharpe",      # sharpe | accuracy | f1 | ...
    n_iter=5               # número de combinaciones a probar
)

In [ ]:
# Obtener un modelo específico
best_model_lstm_longshort = results_longshort.get_model_by_id("LSTM_comb5")
best_model_gru_longshort = results_longshort.get_model_by_id("GRU_comb5")
best_model_cnn_lstm_longshort = results_longshort.get_model_by_id("LSTM_CNN_comb3")
best_model_cnn_gru_longshort = results_longshort.get_model_by_id("GRU_CNN_comb5")

In [ ]:
# Lista de modelos ya entrenados
selected_models = [
    {"name": "LSTM_final", "model": lstm_model},
    {"name": "GRU_final", "model": gru_model},
    {"name": "LSTM_CNN_final", "model": lstm_cnn_model},
    {"name": "GRU_CNN_final", "model": gru_cnn_model},
]
